In [ ]:
import cv2
import dlib
import numpy as np
import tensorflow as tf
from imutils import face_utils

# dlib의 얼굴 검출기와 랜드마크 예측기 초기화
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # dlib의 얼굴 랜드마크 모델

# 눈의 상태를 판별하기 위한 함수
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# 눈이 감겼는지 확인하는 임계값과 프레임 수 설정
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3
COUNTER = 0

# 실시간 웹캠 추론
cap = cv2.VideoCapture(0)
model = tf.keras.models.load_model('drowsy_detection_model.h5')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[42:48]
        rightEye = shape[36:42]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        # 눈의 랜드마크에 사각형을 그림
        for (x, y) in leftEye:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
        for (x, y) in rightEye:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

        # EAR 값 기반으로 눈 상태 판단
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                ear_label = "Closed"
                ear_color = (0, 0, 255)
        else:
            COUNTER = 0
            ear_label = "Open"
            ear_color = (0, 255, 0)

        # CNN 기반 눈 상태 판단
        eye_img = frame[rect.top():rect.bottom(), rect.left():rect.right()]
        eye_img = cv2.resize(eye_img, (224, 224))
        eye_img = eye_img.astype('float') / 255.0
        eye_img = np.expand_dims(eye_img, axis=0)
        
        cnn_prediction = model.predict(eye_img)
        
        if cnn_prediction < 0.5:
            cnn_label = "Closed"
            cnn_color = (0, 0, 255)
        else:
            cnn_label = "Open"
            cnn_color = (0, 255, 0)

        # 최종 판단 (EAR과 CNN 결과를 모두 고려)
        if ear_label == "Closed" or cnn_label == "Closed":
            final_label = "Closed"
            final_color = (0, 0, 255)
        else:
            final_label = "Open"
            final_color = (0, 255, 0)

        # 얼굴과 눈 상태 표시
        cv2.putText(frame, final_label, (rect.left(), rect.top() - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, final_color, 2)
        cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), final_color, 2)

    cv2.imshow('Drowsiness Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
